<h2>1. Preparation</h2>

In [1]:
import pickle 
import pandas
import numpy
import time
import cv2
import random

from sklearn import metrics, preprocessing 
from sklearn.cluster import AgglomerativeClustering, KMeans, Birch
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

from variables import pickle_path, temp_path

In [2]:
CLUSTER_EXECUTION = 'RELOAD'
DATA_SCALER_AND_PCA = 'RELOAD'
N_RACES = pickle.load(open(pickle_path+"N_FILTERED_RACES_last_execution.p", "rb" ))
N_RACES

10

In [3]:
df_train = pickle.load(open(pickle_path + "df_train_%s_races.p"%N_RACES, "rb" ))
df_test = pickle.load(open(pickle_path + "df_test_%s_races.p"%N_RACES, "rb" ))
dict_pictures_features_caracs = pickle.load(open(pickle_path+"dict_pictures_features_caracs_%s_races.p"%N_RACES, "rb" ))

In [4]:
# randomly choose 10% features
L_indexes = list(range(0, len(df_train)))
cluster_train_thresold = int(len(df_train) * 0.1)
L_cluster_indexes = random.sample(L_indexes, cluster_train_thresold)
L_cluster_indexes = numpy.sort(L_cluster_indexes)
L_cluster_indexes

array([     27,      65,      66, ..., 2091040, 2091044, 2091049])

In [5]:
df_train_sample = df_train.copy()
df_train_sample = df_train_sample.loc[L_cluster_indexes]
df_train_sample

,B_value,G_value,R_value,feature_dominant_color,feature_id,picture_id,picture_name,race,0,1,...,127,feature_dominant_color_balanced,feature_dominant_color_blue,feature_dominant_color_gb,feature_dominant_color_green,feature_dominant_color_rb,feature_dominant_color_red,feature_dominant_color_rg,created_columns,dict_mapping
27,16,9,12,blue,27,0,basenji_1013.jpg,basenji,57.0,41.0,...,3.0,0,1,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
65,26,23,25,balanced,65,0,basenji_1013.jpg,basenji,11.0,30.0,...,9.0,1,0,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
66,3,1,1,blue,66,0,basenji_1013.jpg,basenji,23.0,22.0,...,1.0,0,1,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
74,26,23,25,balanced,74,0,basenji_1013.jpg,basenji,2.0,8.0,...,1.0,1,0,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
90,5,0,4,blue,90,0,basenji_1013.jpg,basenji,16.0,12.0,...,38.0,0,1,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
104,7,4,6,blue,104,0,basenji_1013.jpg,basenji,22.0,49.0,...,60.0,0,1,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
109,11,1,7,blue,109,0,basenji_1013.jpg,basenji,14.0,9.0,...,6.0,0,1,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
111,4,0,5,red,111,0,basenji_1013.jpg,basenji,37.0,37.0,...,36.0,0,0,0,0,0,1,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
116,5,1,6,red,116,0,basenji_1013.jpg,basenji,18.0,11.0,...,21.0,0,0,0,0,0,1,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...
123,29,28,30,balanced,123,0,basenji_1013.jpg,basenji,14.0,2.0,...,24.0,1,0,0,0,0,0,0,"[feature_dominant_color_balanced, feature_domi...",{'feature_dominant_color': ['feature_dominant_...


<h2>2. K-Means</h2>

In [6]:
L_interest_features = ['R_value', 'G_value', 'B_value']
L_interest_features += df_train['created_columns'][0].copy()
L_interest_features += list(range(0, 128))
L_interest_features

['R_value',
 'G_value',
 'B_value',
 'feature_dominant_color_balanced',
 'feature_dominant_color_blue',
 'feature_dominant_color_gb',
 'feature_dominant_color_green',
 'feature_dominant_color_rb',
 'feature_dominant_color_red',
 'feature_dominant_color_rg',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127]

In [7]:
X_sample = df_train_sample[L_interest_features].values
if DATA_SCALER_AND_PCA == 'RELOAD':  
    std_scale = StandardScaler().fit(X_sample)
    pickle.dump(std_scale, open(pickle_path+"data_scaler.p", "wb"))

if DATA_SCALER_AND_PCA == 'LOAD':
    std_scale = pickle.load(open(pickle_path+"data_scaler.p", "rb" ))

X_scaled_sample = std_scale.transform(X_sample)
if DATA_SCALER_AND_PCA == 'RELOAD':  
    pca = PCA(n_components=80)
    pca.fit(X_scaled_sample)
    pickle.dump(pca, open(pickle_path+"pca_fitted.p", "wb"))

if DATA_SCALER_AND_PCA == 'LOAD':
    pca = pickle.load(open(pickle_path+"pca_fitted.p", "rb" ))

print(pca.explained_variance_ratio_.cumsum())
X_PCA_sample = pca.transform(X_scaled_sample)

C:\Users\Allen\Desktop\dogs\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Allen\Desktop\dogs\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


[0.07401271 0.13640914 0.18432074 0.23156219 0.27472074 0.31355043
 0.34437477 0.37495971 0.40447723 0.42874815 0.45109201 0.47192102
 0.49149219 0.50938263 0.52639572 0.5431742  0.55915679 0.57393321
 0.58804686 0.60193773 0.61572622 0.62850034 0.6404347  0.65168947
 0.66249556 0.67176124 0.68099859 0.69014507 0.6990619  0.70784049
 0.71581092 0.72370537 0.73131875 0.73880791 0.74609439 0.75335029
 0.76050304 0.76762611 0.77457495 0.78148213 0.78801873 0.79439063
 0.80070357 0.80692388 0.81298402 0.81890428 0.82449776 0.82999972
 0.83541193 0.84061432 0.84556026 0.85037727 0.85513575 0.85972168
 0.86399429 0.8682147  0.87223287 0.87621892 0.88007199 0.88378978
 0.88743756 0.89105191 0.89452192 0.89788472 0.90108342 0.90420144
 0.9071979  0.91018998 0.91304958 0.91586369 0.91853628 0.92119102
 0.92380006 0.92632662 0.92877895 0.93107487 0.93332875 0.93555668
 0.9377628  0.93993358]


In [8]:
# cluster using k-means
if CLUSTER_EXECUTION == 'RELOAD':
    
    X_to_fit = X_PCA_sample
#     %pylab inline
    dict_clustering_models = {'GaussianMixture':{}, 'KMeans':{}, 'Birch':{}}
    dict_labels = {'GaussianMixture':{}, 'KMeans':{}, 'Birch':{}}
    dict_clusters_centers = {'GaussianMixture':{}, 'KMeans':{}, 'Birch':{}}
    
    for num_clusters in [50, 100, 500, 1000]:
        start_time = time.time()
        print("For %s clusters:"%num_clusters)
        
        # k-means clustering
        init_fitting_time = time.time()
        KM = KMeans(n_clusters=num_clusters, random_state =42, init='k-means++')
        KM = KM.fit(X_to_fit)
        labels_KM = KM.labels_
        cluster_centers_KM = KM.cluster_centers_
        print("STEP TIME: %ss"%(time.time() - init_fitting_time))
        
        dict_clustering_models['KMeans'][num_clusters] = KM
        dict_labels['KMeans'][num_clusters] = labels_KM
        dict_clusters_centers['KMeans'][num_clusters] = cluster_centers_KM
        
        pickle.dump(dict_clustering_models,open(pickle_path + "dict_clustering_models_%s_%s_races.p"%(num_clusters, N_RACES), "wb"))
        pickle.dump(dict_labels,open(pickle_path + "dict_labels_%s_%s_races.p"%(num_clusters, N_RACES), "wb"))
        pickle.dump(dict_clusters_centers,open(pickle_path + "dict_cluster_centers_%s_%s_races.p"%(num_clusters, N_RACES), "wb"))

For 50 clusters:
STEP TIME: 317.4265525341034s
For 100 clusters:
STEP TIME: 684.5575170516968s
For 500 clusters:
STEP TIME: 2596.449887275696s
For 1000 clusters:
STEP TIME: 3413.0959873199463s


<h2>3. Clustering All</h2>

In [9]:
L_all_races = list(numpy.unique(df_train['race']))
labelencoder = LabelEncoder()
L_encoded_races = labelencoder.fit_transform(L_all_races)
pickle.dump(labelencoder,open(pickle_path + "labelencoder_%s_races.p"%N_RACES, "wb"))

In [10]:
# PCA on train data
X_train = df_train[L_interest_features].values
X_scaled_train  = std_scale.transform(X_train)
X_PCA_train  = pca.transform(X_scaled_train)

# PCA on test data
X_test = df_test[L_interest_features].values
X_scaled_test  = std_scale.transform(X_test)
X_PCA_test  = pca.transform(X_scaled_test)

C:\Users\Allen\Desktop\dogs\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Allen\Desktop\dogs\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [11]:
def values_to_col(myDataFrame, myColumnList, boolOldName):

    created_columns = []
    dict_mapping = {}
    for column in myColumnList:
        # filling missing values
        myDataFrame[column].fillna('none', inplace=True)
        newFeatures = []
        corpus = myDataFrame[column]
        vectorizer = CountVectorizer(min_df=1, max_df=1.0)

        # construction
        X = vectorizer.fit_transform(corpus).toarray()
        feature_names = vectorizer.get_feature_names()

        for feature in feature_names:
            if boolOldName:
                newFeatureName = '%s_%s' % (column, feature)
            else:
                newFeatureName = feature

            newFeatures.append(newFeatureName)
            created_columns.append(newFeatureName)

            if column in dict_mapping:
                dict_mapping[column].append(newFeatureName)
            else:
                dict_mapping[column] = [newFeatureName]

        featuredFrame = pandas.DataFrame(X, columns=newFeatures)
        myDataFrame = pandas.concat([myDataFrame, featuredFrame], axis=1, join_axes=[featuredFrame.index])
        myDataFrame['created_columns'] = [created_columns] * len(myDataFrame)
        myDataFrame['dict_mapping'] = [dict_mapping] * len(myDataFrame)

    return myDataFrame

In [12]:
for num_clusters in [50, 100, 500, 1000]:
    start_time = time.time()
    print('%s races and %s clusters:'%(N_RACES, num_clusters))
    N_FINAL_CLUSTERS = num_clusters
    FINAL_CLUSTERING = 'KMeans'
    
    dict_labels = pickle.load(open(pickle_path + "dict_labels_%s_%s_races.p"%(N_FINAL_CLUSTERS, N_RACES), "rb" ))
    labels_KM = dict_labels['KMeans'][N_FINAL_CLUSTERS]
    cluster_centers_KM = pickle.load(open(pickle_path + "dict_cluster_centers_%s_%s_races.p"%(N_FINAL_CLUSTERS,N_RACES), "rb" ))
    dict_clustering_models = pickle.load(open(pickle_path + "dict_clustering_models_%s_%s_races.p"%(N_FINAL_CLUSTERS,N_RACES), "rb" ))
    
    df_train_sample['cluster'] = dict_labels[FINAL_CLUSTERING][N_FINAL_CLUSTERS]
    
    # apply to the all data
    clustering_model = dict_clustering_models[FINAL_CLUSTERING][N_FINAL_CLUSTERS]
    
    # predict train data cluster and set cluster label
    L_train_clusters = clustering_model.predict(X_PCA_train)
    df_train['cluster'] = L_train_clusters
    df_train['cluster'] = df_train['cluster'].astype(str)
    
    # predict test data cluster and set cluster label
    L_test_clusters = clustering_model.predict(X_PCA_test)
    df_test['cluster'] = L_test_clusters
    df_test['cluster'] = df_test['cluster'].astype(str)
    
    L_all_races = list(numpy.unique(df_train['race']))
    
    # creating bag of visual words for each race
    for race in L_all_races:
        print(race)
        sub_df_train = df_train[['picture_name','cluster']][df_train['race'] == race].copy()
        sub_df_test = df_test[['picture_name','cluster']][df_test['race'] == race].copy()

        sub_df_train.index = range(0,len(sub_df_train))
        sub_df_test.index = range(0,len(sub_df_test))

        sub_df_train = values_to_col(sub_df_train,['cluster'],True)
        sub_df_test = values_to_col(sub_df_test,['cluster'],True)

        focus_columns_train = sub_df_train['created_columns'][0]
        focus_columns_train+=['picture_name']

        focus_columns_test = sub_df_test['created_columns'][0]
        focus_columns_test+=['picture_name']

        sub_df_train = sub_df_train[focus_columns_train].groupby('picture_name').sum()
        sub_df_test = sub_df_test[focus_columns_test].groupby('picture_name').sum()

        sub_df_train_name = 'bag_of_visual_words_train_%s_races_%s_clusters'%(race,num_clusters)
        pickle.dump(sub_df_train,open(temp_path + sub_df_train_name + ".p", "wb"))

        sub_df_test_name = 'bag_of_visual_words_test_%s_races_%s_clusters'%(race,num_clusters)
        pickle.dump(sub_df_test,open(temp_path + sub_df_test_name + ".p", "wb"))
    
    focus_columns = ['picture_name']
    focus_columns += ["cluster_%s"%i for i in range(0, N_FINAL_CLUSTERS)]
    
    df_model_train = pandas.DataFrame(columns = focus_columns)
    df_model_test = df_model_train.copy()
    
    for race in L_all_races:
        sub_df_train_name = 'bag_of_visual_words_train_%s_races_%s_clusters'%(race, num_clusters)
        sub_df_test_name = 'bag_of_visual_words_test_%s_races_%s_clusters'%(race, num_clusters)

        sub_df_train = pickle.load(open(temp_path + sub_df_train_name + ".p", "rb" ))
        sub_df_test = pickle.load(open(temp_path + sub_df_test_name + ".p", "rb" ))

        sub_df_train['picture_name'] = sub_df_train.index
        sub_df_test['picture_name'] = sub_df_test.index

        df_model_train = df_model_train.append(sub_df_train)
        df_model_test = df_model_test.append(sub_df_test)

    df_model_train.fillna(0, inplace=True)
    df_model_test.fillna(0, inplace=True)

    df_model_train = df_model_train[focus_columns]
    df_model_test = df_model_test[focus_columns]
    
    # normalize the picture/cluster distribution for each train and test dataframes
    orig_picture_names_train = df_model_train['picture_name'].copy()
    orig_picture_names_test = df_model_test['picture_name'].copy()
    
    focus_columns_with_picture_name = focus_columns.copy()
    focus_columns.remove('picture_name')
    
    # normalize the bag of visual words for train data
    start_time = time.time()
    for row in df_model_train.index:
        row_sum = numpy.sum(df_model_train.loc[row,focus_columns])
        df_model_train.loc[row,focus_columns] = df_model_train.loc[row,focus_columns]/row_sum
    df_model_train['picture_name'] = orig_picture_names_train
    df_model_train = df_model_train[focus_columns_with_picture_name]
    
    # normalize the bag of visual words for test data
    for row in df_model_test.index:
        row_sum = numpy.sum(df_model_test.loc[row,focus_columns])
        df_model_test.loc[row,focus_columns] = df_model_test.loc[row,focus_columns]/row_sum
    df_model_test['picture_name'] = orig_picture_names_test
    df_model_test = df_model_test[focus_columns_with_picture_name]
    
    df_race_picture_train = df_train[['race','picture_name']].copy()
    df_race_picture_train.drop_duplicates(subset=['race','picture_name'], inplace=True)
    df_race_picture_test = df_test[['race','picture_name']].copy()
    df_race_picture_test.drop_duplicates(subset=['race','picture_name'], inplace=True)
    
    df_model_train = pandas.merge(df_model_train, df_race_picture_train, how='left', on='picture_name')
    df_model_test = pandas.merge(df_model_test, df_race_picture_test, how='left', on='picture_name')
    
    # encode race col
    df_model_train['encoded_race'] = df_model_train['race'].apply(lambda x : labelencoder.transform([x])[0])
    df_model_test['encoded_race'] = df_model_test['race'].apply(lambda x : labelencoder.transform([x])[0])
    
    pickle.dump(df_model_train,open(pickle_path+"df_model_train_with_%s_clusters_%s_races.p"%(N_FINAL_CLUSTERS,N_RACES), "wb"))
    pickle.dump(df_model_test,open(pickle_path+"df_model_test_with_%s_clusters_%s_races.p"%(N_FINAL_CLUSTERS,N_RACES), "wb"))
    
    print("STEP TIME: %ss"%(time.time() - start_time))

10 races and 50 clusters:
basenji
chesapeake_bay_retriever
collie
irish_setter
italian_greyhound
mexican_hairless
miniature_poodle
pug
schipperke
tibetan_terrier


C:\Users\Allen\Desktop\dogs\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


STEP TIME: 18.333671808242798s
10 races and 100 clusters:
basenji
chesapeake_bay_retriever
collie
irish_setter
italian_greyhound
mexican_hairless
miniature_poodle
pug
schipperke
tibetan_terrier


C:\Users\Allen\Desktop\dogs\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


STEP TIME: 30.9891574382782s
10 races and 500 clusters:
basenji
chesapeake_bay_retriever
collie
irish_setter
italian_greyhound
mexican_hairless
miniature_poodle
pug
schipperke
tibetan_terrier


C:\Users\Allen\Desktop\dogs\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


STEP TIME: 137.81572461128235s
10 races and 1000 clusters:
basenji
chesapeake_bay_retriever
collie
irish_setter
italian_greyhound
mexican_hairless
miniature_poodle
pug
schipperke
tibetan_terrier


C:\Users\Allen\Desktop\dogs\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


STEP TIME: 272.3052728176117s
